In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
mnist = input_data.read_data_sets("../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/",one_hot=True)

Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
tf.reset_default_graph() 

In [77]:
# DEFINE parameters
num_inputs = 10
neurons_hid1 = 5
neurons_hid2 = 3
neurons_hid3 = 2

learning_rate = 0.00003

In [78]:
# PINV(PSEUDO-INVERSE) function
def pinv(a, rcond=1e-15):
    s, u, v = tf.svd(a)
    # Ignore singular values close to zero to prevent numerical overflow
    limit = rcond * tf.reduce_max(s)
    non_zero = tf.greater(s, limit)

    reciprocal = tf.where(non_zero, tf.reciprocal(s), tf.zeros(s.shape))
    lhs = tf.matmul(v, tf.diag(reciprocal))
    return tf.matmul(lhs, u, transpose_b=True)

In [79]:
# TRANSPOSE OR PINV
tie_weight = tf.transpose #pinv

In [80]:
## PLACEHOLDER DEFINATION
X = tf.placeholder(tf.float32, shape=[None, num_inputs])

In [81]:
## WEIGHTS DEFINATION
initializer = tf.variance_scaling_initializer()

w1 = tf.Variable(np.load('./save/weights/lw1.npy'))
w1_ = tie_weight(w1)

w2 = tf.Variable(np.load('./save/weights/lw2.npy'))
w2_ = tie_weight(w2)

w3 = tf.Variable(initializer([neurons_hid2, neurons_hid3]), dtype=tf.float32)
w3_ = tie_weight(w3)

In [82]:
# ACTIVATION FUNCTION  [ lambda X:X  <OR>  tf.nn.relu  ]
act_func = lambda X:X #tf.nn.relu

In [83]:
# LAYER MODELING OF :NN
hid_layer1 = act_func(tf.matmul(X, w1))
hid_layer2 = act_func(tf.matmul(hid_layer1, w2))

hid_layer3 = act_func(tf.matmul(hid_layer2, w3))

hid_layer2_= act_func(tf.matmul(hid_layer3, w3_))
hid_layer1_= act_func(tf.matmul(hid_layer2_, w2_))
output_layer = tf.nn.softmax(tf.matmul(hid_layer1_, w1_))

In [84]:
# LOSS FUNCTIONS
# loss = tf.reduce_mean(tf.square(output_layer - X))
# loss = tf.reduce_mean(tf.abs(output_layer - X))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=X,logits=output_layer))

In [85]:
# TRAINING
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss, var_list=[w3])

In [86]:
# PREPARATION
init = tf.global_variables_initializer()
saver = tf.train.Saver() 

In [88]:
num_epochs = 100
batch_size = 150

try:
    with tf.Session() as sess:
        sess.run(init)

        # Epoch == Entire Training Set
        for epoch in range(num_epochs):
            num_batches = mnist.train.num_examples // batch_size

            # 150 batch size
            for iteration in range(num_batches):
                X_batch, y_batch = mnist.train.next_batch(batch_size)
                sess.run(train, feed_dict={X: y_batch})

            training_loss = loss.eval(feed_dict={X: y_batch})   
            print("Epoch {} Complete. Training Loss: {}".format(epoch,training_loss))
            saver.save(sess, "./save/label_ae_3.ckpt")
            
            
            
except KeyboardInterrupt:
    print('Keyboard Interrupted')
finally:
    print('Finished Training the Model')

Epoch 0 Complete. Training Loss: 1.9471555948257446
Epoch 1 Complete. Training Loss: 1.9665710926055908
Epoch 2 Complete. Training Loss: 1.864491581916809
Epoch 3 Complete. Training Loss: 1.8764228820800781
Epoch 4 Complete. Training Loss: 1.9083211421966553
Epoch 5 Complete. Training Loss: 1.9254591464996338
Epoch 6 Complete. Training Loss: 1.9468547105789185
Epoch 7 Complete. Training Loss: 1.8455122709274292
Epoch 8 Complete. Training Loss: 1.8190712928771973
Epoch 9 Complete. Training Loss: 1.899095058441162
Epoch 10 Complete. Training Loss: 1.8678102493286133
Epoch 11 Complete. Training Loss: 1.8378230333328247
Epoch 12 Complete. Training Loss: 1.8705426454544067
Epoch 13 Complete. Training Loss: 1.828100323677063
Epoch 14 Complete. Training Loss: 1.8310855627059937
Epoch 15 Complete. Training Loss: 1.8672418594360352
Epoch 16 Complete. Training Loss: 1.870818018913269
Epoch 17 Complete. Training Loss: 1.8299062252044678
Epoch 18 Complete. Training Loss: 1.837077021598816
Epoch 19

In [89]:
num_test_labels = 10
start_point = 15
with tf.Session() as sess:
    
    saver.restore(sess,"./save/label_ae_3.ckpt")
    
    results,compressed = sess.run([output_layer, hid_layer3],
                                  feed_dict={X:mnist.test.labels[start_point:num_test_labels+start_point]})
    weight3 = w3.eval()
    
    matches = tf.equal(tf.argmax(output_layer, 1), tf.argmax(X,1))
    acc = tf.reduce_mean(tf.cast(matches, tf.float32))
    print('TRAIN ACCURACY: ')
    print (sess.run(acc, feed_dict={X:mnist.train.labels}))

    print('TEST ACCURACY: ')
    print (sess.run(acc, feed_dict={X:mnist.test.labels}))
    print('\n')

INFO:tensorflow:Restoring parameters from ./save/label_ae_3.ckpt
TRAIN ACCURACY: 
1.0
TEST ACCURACY: 
1.0




In [90]:
compressed.shape

(10, 2)

In [ ]:
# ORIGINAL VS RECONSTRUCTED
f, a = plt.subplots(3, num_test_labels, figsize=(20, 6))
for i in range(start_point,num_test_labels+start_point):
    j = i-start_point
    a[0][j].imshow(np.reshape(mnist.test.labels[i], (10, 1)))
    a[1][j].imshow(np.reshape(results[j], (10, 1)))
    a[2][j].imshow(np.reshape(compressed[j], (2, 1)))
    print(mnist.test.labels[i])
    np.set_printoptions(precision=4, suppress=True)
    print(results[j])

In [93]:
weight3

array([[ 0.0463, -0.9424],
       [ 0.95  ,  0.9727],
       [ 1.1914, -0.7222]], dtype=float32)

In [94]:
print(weight3.shape)
np.save('./save/weights/lw3.npy',weight3)
#wt1 = np.load('./save/weights/w1.npy')

(3, 2)
